In [ ]:
%pylab inline
import pandas as pd
import statsmodels.api as sm

In [ ]:
SUBJECTS = ['RS02','RS03','RS04']

Neeraj's super clear naming scheme:

* 1 = familiarisation
* 2 = baseline
* 3 = day 1 training rotation
* 4 = day 1 end recall
* 5 = day 2 recall
* 6 = day 2 retraining


Let's take the following sets of trials (direction at vmax) from each subject:

* `recall1` End day 1 no feedback   (session 4)
* `trainnof1` Train day 1 no feedback trials (session 3 but no Feedback only)
* `trainfb1` Train day 1 last feedback trials (session 3 but with Feedback and taking only the last N trials)

And also for everyone:

* `recall2` Recall day 2 no feedback (session 5)

In [ ]:
PREDICTORS = ['recall1','trainnof1','trainfb1'] # these are the "samples" we use as predictors

In [ ]:
TARGET = 'recall2'  # what we are trying to predict

In [ ]:
# How many trials to take from the end of the day1 feedback trials 
N_TRIALS = 20

In [ ]:
alldata = []
samples = {} # for each subject we put a set of samples
for subj in SUBJECTS:
    tab = pd.read_csv('{}_data.csv'.format(subj))
    alldata.append(tab)
    samples[subj]={
        "trainnof1":list(tab[ (tab['session']==3) & (tab['trialtype']=='noFeedback') ]['handAngle']),
        "trainfb1": list(tab[ (tab['session']==3) & (tab['trialtype']=='feedback') ]['handAngle'])[-N_TRIALS:],
        "recall1":list(tab[ tab['session']==4 ]['handAngle']),
        "recall2":list(tab[ tab['session']==5 ]['handAngle']) 
    }
alldata = pd.concat(alldata)
alldata.to_csv('tmp.csv')

In [ ]:
#samples

In [ ]:
alldata.head()

# First approach: taking average
For each predictor, we can take the average, and try to predict the movements during the target phase.

In [ ]:
preds = []
for subj,values in samples.items():
    d = {"subject":subj}
    for predictor in PREDICTORS:
        d[predictor]=mean(samples[subj][predictor])
    preds.append(d)
preds = pd.DataFrame(preds)

In [ ]:
preds

In [ ]:
targ = []
for subj,values in samples.items():
    d = pd.DataFrame()
    d[TARGET]=values[TARGET]
    d["subject"]=subj
    targ.append(d)
targ = pd.concat(targ)

In [ ]:
#targ

In [ ]:
alldat = pd.merge(preds,targ)

In [ ]:
alldat.head()

In [ ]:
PREDICTORS

In [ ]:
predictor = 'recall1'

In [ ]:
Y = alldat[TARGET] 
X = alldat[predictor]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
print(results.summary())

In [ ]:
results.params

In [ ]:
for predictor in PREDICTORS:
    Y = alldat[TARGET] 
    X = alldat[predictor]
    #X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    print(predictor)
    print(list(results.params))
    print(list(results.tvalues))
    print()
    #print("{} {}".format(predictor,results.params))
    #print(results.summary())

In [ ]:
import statsmodels.api as sm

In [ ]:
# TODO: 